In [1]:
from dataset import normalizate

norm = normalizate("datas/negativa", "datas/negativa normalizada")

In [2]:
norm.normalizar()

Normalizando arquivos de áudio: 100%|██████████| 3747/3747 [08:18<00:00,  7.52arquivo/s]


In [3]:
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.preprocessing import StandardScaler
import os
class WakeWordCNN(nn.Module):
    def __init__(self, num_mfccs, num_frames_per_sample):
        super(WakeWordCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3), padding='same')
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3, 3), padding='same')
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3, 3), padding='same')
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=(2, 2))

        # Recalcula _to_linear para garantir que seja o mesmo do treinamento
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, num_mfccs, num_frames_per_sample)
            x = self.pool1(F.relu(self.bn1(self.conv1(dummy_input))))
            x = self.pool2(F.relu(self.bn2(self.conv2(x))))
            x = self.pool3(F.relu(self.bn3(self.conv3(x))))
            self._to_linear = x.shape[1] * x.shape[2] * x.shape[3]

        self.fc1 = nn.Linear(self._to_linear, 256)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x.squeeze(1)


In [4]:
DATASET_ROOT_DIR = "datas" 

SEGMENT_DURATION_SECONDS = 3


SAMPLERATE = 16000

N_MFCC = 40     
N_FFT = 400       
HOP_LENGTH = 160  

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {DEVICE}")

Usando dispositivo: cuda


In [5]:
from dataset import WakeWordDataset
from joblib import dump

dummy_waveform = torch.randn(1, int(SAMPLERATE * SEGMENT_DURATION_SECONDS))
mfcc_test_transform = torchaudio.transforms.MFCC(
    sample_rate=SAMPLERATE, n_mfcc=N_MFCC,
    melkwargs={"n_fft": N_FFT, "hop_length": HOP_LENGTH, "n_mels": N_MFCC}
)
dummy_mfccs = mfcc_test_transform(dummy_waveform)
NUM_FRAMES_PER_SAMPLE = dummy_mfccs.shape[2] 

print(f"Número de frames MFCC por amostra de {SEGMENT_DURATION_SECONDS}s: {NUM_FRAMES_PER_SAMPLE}")

# 1. Cria a instância do Dataset
dataset = WakeWordDataset(
    root_dir=DATASET_ROOT_DIR,
    segment_duration_seconds=SEGMENT_DURATION_SECONDS,
    samplerate=SAMPLERATE,
    n_mfcc=N_MFCC,
    n_fft=N_FFT,
    hop_length=HOP_LENGTH
)
print(f"Total de amostras no dataset: {len(dataset)}")

# 2. Ajusta o scaler (normalizador) para as MFCCs

dataset.fit_scaler()
dump(dataset.scaler, "scaler.pkl")

# 3. Divide o dataset em treino e validação
train_size = int(0.8 * len(dataset)) # 80% para treino
val_size = len(dataset) - train_size # 20% para validação
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f"Tamanho do dataset de treino: {len(train_dataset)}")
print(f"Tamanho do dataset de validação: {len(val_dataset)}")

# 4. Cria os DataLoaders para iterar sobre os dados em lotes (batches)
BATCH_SIZE = 32 # Tamanho do lote
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=os.cpu_count() // 2 or 1)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=os.cpu_count() // 2 or 1)

print(f"Número de workers para DataLoader: {os.cpu_count() // 2 or 1}")


Número de frames MFCC por amostra de 3s: 301
Carregadas 72 amostras positivas.
Carregadas 7487 amostras negativas.
Total de amostras no dataset: 7559
Ajustando normalizador (scaler) para as MFCCs...
Normalizador ajustado.
Tamanho do dataset de treino: 6047
Tamanho do dataset de validação: 1512
Número de workers para DataLoader: 8


In [6]:


# 6. Instancia o Modelo CNN
model = WakeWordCNN(N_MFCC, NUM_FRAMES_PER_SAMPLE).to(DEVICE)


In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device, model_save_path):
    best_val_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train() # Define o modelo para modo de treinamento
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for i, (mfccs, labels) in enumerate(train_loader):
            mfccs = mfccs.to(device)
            labels = labels.to(device)

            # Zera os gradientes
            optimizer.zero_grad()

            # Passagem para frente
            outputs = model(mfccs)
            loss = criterion(outputs, labels)

            # Passagem para trás e otimização
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * mfccs.size(0)

            # Calcula a acurácia
            # Saída do modelo são logits, sigmoid para obter probabilidades
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / total_samples
        epoch_accuracy = correct_predictions / total_samples
        print(f"Epoch {epoch+1}/{num_epochs} - Treino Loss: {epoch_loss:.4f}, Treino Acurácia: {epoch_accuracy:.4f}")

        # --- Validação ---
        model.eval() # Define o modelo para modo de avaliação (desativa dropout, etc.)
        val_running_loss = 0.0
        val_correct_predictions = 0
        val_total_samples = 0
        all_val_labels = []
        all_val_predictions = []

        with torch.no_grad(): # Desativa o cálculo de gradientes para validação
            for mfccs, labels in val_loader:
                mfccs = mfccs.to(device)
                labels = labels.to(device)

                outputs = model(mfccs)
                loss = criterion(outputs, labels)
                val_running_loss += loss.item() * mfccs.size(0)

                predicted = (torch.sigmoid(outputs) > 0.5).float()
                val_correct_predictions += (predicted == labels).sum().item()
                val_total_samples += labels.size(0)

                all_val_labels.extend(labels.cpu().numpy())
                all_val_predictions.extend(predicted.cpu().numpy())

        val_loss = val_running_loss / val_total_samples
        val_accuracy = val_correct_predictions / val_total_samples
        print(f"Epoch {epoch+1}/{num_epochs} - Validação Loss: {val_loss:.4f}, Validação Acurácia: {val_accuracy:.4f}")

        # Calcula métricas adicionais de validação
        val_precision = precision_score(all_val_labels, all_val_predictions, zero_division=0)
        val_recall = recall_score(all_val_labels, all_val_predictions, zero_division=0)
        val_f1 = f1_score(all_val_labels, all_val_predictions, zero_division=0)
        print(f"Validação Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")


        # Salva o melhor modelo com base na acurácia de validação
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), model_save_path)
            print(f"Modelo salvo em '{model_save_path}' com acurácia de validação: {best_val_accuracy:.4f}")

    print("\nTreinamento concluído!")

In [ ]:

LEARNING_RATE = 0.001
NUM_EPOCHS = 20 # Número de vezes que o modelo verá todo o dataset
BATCH_SIZE = 32
num_arquivos = sum(
    1 for f in os.listdir('models/') if os.path.isfile(os.path.join('models/', f))
) + 1
MODEL_SAVE_PATH = f"models/wake_word_model_{num_arquivos}.pth"
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# 7. Inicia o treinamento
train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE, MODEL_SAVE_PATH)

print("\nTreinamento do modelo de Wake Word concluído e melhor modelo salvo!")
print(f"O modelo treinado está salvo em: {MODEL_SAVE_PATH}")

Epoch 1/20 - Treino Loss: 0.2452, Treino Acurácia: 0.9866
Epoch 1/20 - Validação Loss: 0.0318, Validação Acurácia: 0.9934
Validação Precision: 0.8000, Recall: 0.3077, F1-Score: 0.4444
Modelo salvo em 'models/wake_word_model_2.pth' com acurácia de validação: 0.9934
Epoch 2/20 - Treino Loss: 0.0317, Treino Acurácia: 0.9916
Epoch 2/20 - Validação Loss: 0.0253, Validação Acurácia: 0.9914
Validação Precision: 0.5000, Recall: 0.0769, F1-Score: 0.1333
